# Part 4: Improve fairness

In the previous notebook we've seen how we can measure fairness. In this notebook we're going to use a TresholdOptimizer to improve the fairness of our model. We'll cover the following topics:

* Loading the dataset and model
* Using the TresholdOptimizer
* Comparing the performance between the old and the new model

## Loading the dataset and model
Before we work on improving the model, let's load the existing model and the dataset.
We'll perform the following steps:

* First, we load the training and testing dataset. 
* Then, we load the model.

In [3]:
import joblib
import pandas as pd

In [4]:
model = joblib.load('../models/model.bin')

In [5]:
df_train = pd.read_csv('../data/processed/train.csv')
df_test = pd.read_csv('../data/processed/test.csv')

In [21]:
x_test = df_test.drop(['SEX', 'default.payment.next.month'], axis=1)
y_test = df_test['default.payment.next.month']

Previously, our dataset was unbalanced since we didn't spend any time looking at that. If we are going to improve the model with the `TresholdOptimizer` we need to make sure that the input has an equal number of samples producing the output value `0` to the number of samples producing the output value `1`. This is a requirement of the algorithm that's used by the optimizer.

In [39]:
balanced_idx = df_train[y_train == 1].index
balanced_idx = balanced_idx.union(
    y_train[y_train==0].sample(n=balanced_idx.size).index
)

In [41]:
x_train_balanced = df_train.loc[balanced_idx, :].drop(['SEX', 'default.payment.next.month'], axis=1)
y_train_balanced = df_train.loc[balanced_idx, 'default.payment.next.month']

Once we have a prepared dataset, we can start working on optimizing the model using the `TresholdOptimizer`.

## Using the TresholdOptimizer
We can improve existing models using the `TresholdOptimizer`. This component takes the original model and adds a layer of optimization on top of it to equalize the performance difference between different demographic groups. It treats the original model as a black box for this.

There are two methods of optimization that you can use:

* Equalized odds
* Demographic parity

Both are loss functions that are used by the optimizer to minimize the differences in performance between demographic groups. Note that we can't fully fix the problems, since this is a problem where we have to maximize performance (thus minimize model loss) and minimize the differences between demographic groups specified as a set of sensitive features. This min-maxing problem turns out to be a balancing act where we can only reach equilibrium between the disparity metric and performance metric.

In [14]:
from fairlearn.postprocessing import ThresholdOptimizer

In [29]:
optimizer = ThresholdOptimizer(estimator=model, constraints='demographic_parity')

In [43]:
optimizer.fit(x_train_balanced, y_train_balanced, sensitive_features=df_train.loc[balanced_idx, 'SEX'])

## Measuring fairness of the new model
Once we have an improved model, we can look at its fairness and compare it to the old model. First, we'll measure the fairness of the new model.

In [44]:
from fairlearn.widget import FairlearnDashboard

In [45]:
FairlearnDashboard(
    sensitive_features=df_test['SEX'],
    sensitive_feature_names=['SEX'],
    y_true=df_test['default.payment.next.month'],
    y_pred=optimizer.predict(x_test, sensitive_features=df_test['SEX'])
)

FairlearnWidget(value={'true_y': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0…

Then, we're comparing the two models.

In [48]:
comparison = {
    'Original model': model.predict(x_test),
    'TresholdOptimizer': optimizer.predict(x_test, sensitive_features=df_test['SEX'])
}

In [50]:
FairlearnDashboard(
    sensitive_features=df_test['SEX'],
    sensitive_feature_names=['Gender'],
    y_true=df_test['default.payment.next.month'],
    y_pred=comparison
)

FairlearnWidget(value={'true_y': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0…